In [1]:
#==============================================================================================
# 원래 vocab 파일에 신규 vocab 파일에 대해 중복제거해서 추가하기
#==============================================================================================
import konlpy
from konlpy.tag import Mecab
from tqdm.notebook import tqdm
from nltk import FreqDist
import numpy as np
import torch
from transformers import BertTokenizer, DistilBertTokenizer

In [2]:
# bert_tokenizer.ipynb에서 만든 new_vocab 파일을 불러옴.
#new_vocab_out = '../../data11/ai_hub/vocab/tl1-1줄-mecab-30000/vocab.txt'
new_vocab_out = '../../data11/my_corpus/re-moco-corpus2/re-moco-corpus2-20000-vocab-add.txt'

# 2차 불용어 정의
#stopwords=['##다', '##하', '있', '##고', '##로', '##한', '##적', '##되', '##었', '##_']
stopwords=['']

new_vocab = []
with open(new_vocab_out, 'r', encoding='utf-8') as f:
    data = f.read().split('\n')

for word in tqdm(data):
    if not word in stopwords:  # 2차 불용어는 제외
        new_vocab.append(word)
        
print(new_vocab[0:10])

  0%|          | 0/13375 [00:00<?, ?it/s]

['[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]', '"', '#', "'", '0', '1']


In [3]:
# 원본 모델의 vocab 리스트로 읽어옴
#org_vocab_out = './moco-vocab/mdistilbertV1.1/vocab.txt'
org_vocab_out = '../../data11/model/bert/kpfbert/vocab.txt'

org_vocab = []

with open(org_vocab_out, 'r', encoding='utf-8') as f:
    data = f.read().split('\n')
    
for vocab in tqdm(data):
     org_vocab.append(vocab)

print(len(org_vocab))


  0%|          | 0/36440 [00:00<?, ?it/s]

36440


In [4]:
import re
start_vocab_id = 2059    # ***추가할 vocab.txt 를 열어서, 추가 시작할 번호 지정 
vocab_len= 10000 # *** 새롭게 추가할 vocab 수 

# 신규 vocab 리스트에서 원본 vocab 리스트 중복값을 제거

# 신규 vocab 리스트에서 원본 vocab 리스트 중복값을 제거
#temp = list(set(new_vocab) - set(org_vocab)) #순서 보존이 안됨
    
#또는
# 신규 vocab 리스트에서 원본 vocab 리스트 중복값을 제거print(org_vocab[1111:1115])
print('중복제거 시작==>')
s = set(org_vocab)
temp = [x for x in tqdm(new_vocab[start_vocab_id:]) if x not in s] #순서 보존됨    

vocablist = org_vocab + temp[:vocab_len]  #원본 리스트 + 중복제가한 신규리스트 2개의 리스트를 합침

print(len(vocablist))
print(vocablist[1111:1115])

중복제거 시작==>


  0%|          | 0/11315 [00:00<?, ?it/s]

43889
['假', '偉', '偏', '停']


In [5]:
# 신규 vocab에서 원본에 중복제거한 목록을 파일로 저장
outfilepath2 = '../../data11/model/bert/kpfbert/re-moco-corpus2-20000-vocab-add-new.txt'
with open(outfilepath2, 'w') as f:
    f.write('\n'.join(temp))

In [6]:
# 원본 vocab+중복제거 신규 vocab 을 리스트를 파일로 저장
outfilepath = '../../data11/model/bert/kpfbert/add-re-moco-corpus2-20000-vocab-add-new.txt'
with open(outfilepath, 'w') as f:
    #f.write('\n'.join(vocablist[:159547])) #159547개만 저장
    f.write('\n'.join(vocablist))  #모두 저장
    
print('vocab 파일"{}" 추가 성공!. 출력 vocab 파일 : "{}"'.format(new_vocab_out, outfilepath))

vocab 파일"../../data11/my_corpus/re-moco-corpus2/re-moco-corpus2-20000-vocab-add.txt" 추가 성공!. 출력 vocab 파일 : "../../data11/model/bert/kpfbert/add-re-moco-corpus2-20000-vocab-add-new.txt"


In [7]:
from transformers import BertTokenizer

#outfilepath = './moco-vocab/moco-corpus-kowiki2022-nouns-42000-Deduplication-add-en-159552.txt'
# BertTokenizer 로 새롭게 저장된 vocalfile 불러옴
tokenizer = BertTokenizer(vocab_file=outfilepath, 
                          strip_accents=False, 
                          do_lower_case=True)


In [8]:
add_special_token = False

if add_special_token:
    # 옵션 : 저장된 add_new_vocab.txt를 불러와서 special 토큰 추가함
    special_tokens=['[BOS]', '[EOS]', '[UNK0]', '[UNK1]', '[UNK2]', '[UNK3]', '[UNK4]', '[UNK5]', '[UNK6]', '[UNK7]', '[UNK8]', '[UNK9]',
                    '[unused0]', '[unused1]', '[unused2]', '[unused3]', '[unused4]', '[unused5]', '[unused6]', '[unused7]', '[unused8]', '[unused9]',]

    special_tokens_dict = {'additional_special_tokens': special_tokens}
    tokenizer.add_special_tokens(special_tokens_dict)

    # special token 체크
    print(tokenizer.all_special_tokens)

In [9]:
# special token 추가한 special vocab을 저장함
import os
OUT_PATH = '../../data11/model/bert/kpfbert/re-moco-corpus2-20000-vocab-add/'
#OUT_PATH = './mdistilbertV2.2'
os.makedirs(OUT_PATH, exist_ok=True)
tokenizer.save_pretrained(OUT_PATH)

('../../data11/model/bert/kpfbert/re-moco-corpus2-20000-vocab-add/tokenizer_config.json',
 '../../data11/model/bert/kpfbert/re-moco-corpus2-20000-vocab-add/special_tokens_map.json',
 '../../data11/model/bert/kpfbert/re-moco-corpus2-20000-vocab-add/vocab.txt',
 '../../data11/model/bert/kpfbert/re-moco-corpus2-20000-vocab-add/added_tokens.json')